# **1. 따릉이 API**
* https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do

In [1]:
import requests
import folium
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 1-1. 데이터 요청하기

In [2]:
targetSite = 'https://www.bikeseoul.com/app/station/getStationRealtimeStatus.do'
request = requests.post(targetSite, data={'stationGrpSeq':'ALL'})
print(request)
# print(request.text)

<Response [200]>


### 1-2. json 데이터 처리하기
* json.loads(): json 타입의 문자열 데이터를 파이썬에서 처리할 수 있도록 변환(딕셔너리로 변환)

In [3]:
bike_json = json.loads(request.text)
# print(bike_json)
print(type(bike_json))

<class 'dict'>


### 1-3. 딕셔너리 타입의 데이터를 데이터프레임으로 변환하기
* json_nomalize(): 딕셔너리의 타입의 데이터를 판다스 데이터프레임으로 변환

In [4]:
bike_df = pd.json_normalize(bike_json, 'realtimeList')
bike_df

,stationId,stationImgFileName,stationName,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,stationSeCd,stationLongitude,stationLatitude,rackTotCnt,mode
0,ST-4,,102. 망원역 1번출구 앞,0,7,10,RAK_002,126.91062927,37.55564880,15,None
1,ST-5,,103. 망원역 2번출구 앞,0,2,0,RAK_002,126.91083527,37.55495071,14,None
2,ST-6,,104. 합정역 1번출구 앞,0,3,0,RAK_002,126.91508484,37.55073929,13,None
3,ST-7,,105. 합정역 5번출구 앞,0,1,0,RAK_002,126.91482544,37.55000687,5,None
4,ST-8,,106. 합정역 7번출구 앞,0,1,0,RAK_002,126.91282654,37.54864502,12,None
...,...,...,...,...,...,...,...,...,...,...,...
2730,ST-3297,,6058. 서울도시건축전시관 옆,0,0,0,RAK_002,126.97684479,37.56653976,10,None
2731,ST-3297,,6058. 서울도시건축전시관 옆,0,0,0,RAK_002,126.97684479,37.56653976,10,None
2732,ST-3276,,6171. 월드빌딩 앞,0,6,0,RAK_002,126.83743286,37.54098129,12,None
2733,ST-3281,,6172. 가양5단지아파트,0,12,0,RAK_002,126.85464478,37.56447983,10,None


In [5]:
bike_df.columns

Index(['stationId', 'stationImgFileName', 'stationName', 'parkingBikeTotCnt',
       'parkingQRBikeCnt', 'parkingELECBikeCnt', 'stationSeCd',
       'stationLongitude', 'stationLatitude', 'rackTotCnt', 'mode'],
      dtype='object')

* stationName: 대여소 이름
* stationId: 고유한 대여소 번호
* stationLongitude: 대여소 경도
* stationLatitude: 대여소 위도
* rackTotCnt: 주차 가능한 전체 자전거 대수
* parkingBikeTotCnt: 주차된 따릉이 총 대수
* parkingQRBikeCnt: 주차된 따릉이 QR형 총 대수
* parkingELECBikeCnt: 주차된 새싹 따릉이 총 대수

In [6]:
bike_df_map = bike_df[['stationName', 'stationId', 'stationLongitude', 'stationLatitude',
                    'rackTotCnt', 'parkingBikeTotCnt', 'parkingQRBikeCnt', 'parkingELECBikeCnt']]
bike_df_map

,stationName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt
0,102. 망원역 1번출구 앞,ST-4,126.91062927,37.55564880,15,0,7,10
1,103. 망원역 2번출구 앞,ST-5,126.91083527,37.55495071,14,0,2,0
2,104. 합정역 1번출구 앞,ST-6,126.91508484,37.55073929,13,0,3,0
3,105. 합정역 5번출구 앞,ST-7,126.91482544,37.55000687,5,0,1,0
4,106. 합정역 7번출구 앞,ST-8,126.91282654,37.54864502,12,0,1,0
...,...,...,...,...,...,...,...,...
2730,6058. 서울도시건축전시관 옆,ST-3297,126.97684479,37.56653976,10,0,0,0
2731,6058. 서울도시건축전시관 옆,ST-3297,126.97684479,37.56653976,10,0,0,0
2732,6171. 월드빌딩 앞,ST-3276,126.83743286,37.54098129,12,0,6,0
2733,6172. 가양5단지아파트,ST-3281,126.85464478,37.56447983,10,0,12,0


In [7]:
# 문제
# 위도, 경도 -> float 변환

# 주차할 수 있는 자전거 대수, 주차된 자전거 총 대수, 주차된 QR자전거 총 대수, 주차된 새싹 자전거 총 대수 -> int
# 파생변수 만들기[total]
# 따릉이 + QR + 새싹

In [8]:
# 위도, 경도 -> float 변환
bike_df_map['stationLongitude'] = bike_df_map['stationLongitude'].astype(float)
bike_df_map['stationLatitude'] = bike_df_map['stationLatitude'].astype(float)

# 주차할 수 있는 자전거 대수, 주차된 자전거 총 대수, 주차된 QR자전거 총 대수, 주차된 새싹 자전거 총 대수 -> int
bike_df_map['rackTotCnt'] = bike_df_map['rackTotCnt'].astype(int)
bike_df_map['parkingBikeTotCnt'] = bike_df_map['parkingBikeTotCnt'].astype(int)
bike_df_map['parkingQRBikeCnt'] = bike_df_map['parkingQRBikeCnt'].astype(int)
bike_df_map['parkingELECBikeCnt'] = bike_df_map['parkingELECBikeCnt'].astype(int)

# 파생변수 만들기[total]
# 따릉이 + QR + 새싹
bike_df_map['total'] = bike_df_map['parkingBikeTotCnt'] + bike_df_map['parkingQRBikeCnt'] + bike_df_map['parkingELECBikeCnt']

In [9]:
bike_df_map.dtypes

stationName            object
stationId              object
stationLongitude      float64
stationLatitude       float64
rackTotCnt              int64
parkingBikeTotCnt       int64
parkingQRBikeCnt        int64
parkingELECBikeCnt      int64
total                   int64
dtype: object

In [10]:
bike_df_map.head()

,stationName,stationId,stationLongitude,stationLatitude,rackTotCnt,parkingBikeTotCnt,parkingQRBikeCnt,parkingELECBikeCnt,total
0,102. 망원역 1번출구 앞,ST-4,126.910629,37.555649,15,0,7,10,17
1,103. 망원역 2번출구 앞,ST-5,126.910835,37.554951,14,0,2,0,2
2,104. 합정역 1번출구 앞,ST-6,126.915085,37.550739,13,0,3,0,3
3,105. 합정역 5번출구 앞,ST-7,126.914825,37.550007,5,0,1,0,1
4,106. 합정역 7번출구 앞,ST-8,126.912827,37.548645,12,0,1,0,1


In [11]:
bike_df_map.shape

(2735, 9)

In [12]:
# {정류소명} 일반:{}대, QR:{}대, 총:{}대
bike_map = folium.Map(location=[bike_df_map['stationLatitude'].mean(),
                                bike_df_map['stationLongitude'].mean()],
                                zoom_start=12)

for index, data in bike_df_map.iterrows():
    popup_str = '{} 일반:{}대, QR:{}대, 새싹:{}대, 총:{}대'.format(
        data['stationName'], data['parkingBikeTotCnt'], data['parkingQRBikeCnt'],
        data['parkingELECBikeCnt'], data['total']
    )
    popup = folium.Popup(popup_str, max_width=600)
    folium.Marker(location=[data['stationLatitude'], data['stationLongitude']],
                  popup=popup).add_to(bike_map)

bike_map

Output hidden; open in https://colab.research.google.com to view.